In [29]:
# Import all required libraries
import pandas as pd
import numpy as np
import pingouin as pg
from pathlib import Path
from scipy import stats
from scipy.stats import spearmanr
import matplotlib.pyplot as plt
import seaborn as sns
import os.path as op
import re
import nibabel as nib

# Additional imports for neuroimaging and atlas processing
from nilearn import datasets, masking, input_data, plotting
from nilearn.maskers import NiftiMapsMasker
from nilearn.connectome import ConnectivityMeasure
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [30]:
# Setup directories and plotting theme
RELI_DIR = Path("dset/derivatives/caps/interrater")
ANNOT_DIR = Path("dset/derivatives/annotations")
FIGURES_DIR = Path("dset/derivatives/figures")
# will use loop later to run over all subjects
OUT_DIR = Path("dset/derivatives/caps")


In [31]:
# Define all runs for each participant and episode
participant_data = {
    "sub-Blossom": {
        "S01E02": [1, 2, 3, 4, 5, 6, 7]      # Alternative name for episode 2
    }
    # Add other participants as needed
}

In [32]:
# Path to the CAP_1 positive weighted spatial mask (z-score weighted)
mask_filename = Path("dset/derivatives/caps/spatial_masks/sub-Blossom_zscore-weighted-0_CAP_1_pos.nii.gz")

print(f"Loading weighted mask: {mask_filename}")

# Load the weighted mask to check its properties
mask_img = nib.load(mask_filename)
mask_data = mask_img.get_fdata()

caps_masker = NiftiMapsMasker(
    maps_img=mask_img,         # your 3D weighted map (or 4D stack of maps)
    standardize=True,          # z-scores the time series
    memory='nilearn_cache',
    mask_type="whole-brain",
    verbose=1
)

Loading weighted mask: dset/derivatives/caps/spatial_masks/sub-Blossom_zscore-weighted-0_CAP_1_pos.nii.gz


In [33]:
# Extract BOLD time series and create z-scored participant matrices
# Note: First 6 TRs from each run will be removed for T1 equilibration

removed_clips_df = pd.read_csv(ANNOT_DIR / "removed_clips_log.csv")

# Initialize storage for participant-level matrices
all_participant_matrices = {}

# Get excluded clips for episode 2
excluded_clips_ep2 = removed_clips_df[removed_clips_df['episode'] == 'S01E02']['episode_position'].tolist()
print(f"Found {len(excluded_clips_ep2)} excluded clips for Episode 2: {excluded_clips_ep2[:10]}...")  # Show first 10

for sub_id, episodes in participant_data.items():
    print(f"\n{'='*60}")
    print(f"PROCESSING {sub_id}")
    print(f"{'='*60}")
    
    participant_timeseries = []
    all_clip_positions = []  # Track which episode position each TR corresponds to
    
    for episode_key, run_numbers in episodes.items():
        # Handle both episode naming conventions
        if episode_key.startswith('S01E'):
            ep_num = int(episode_key[-2:])  # Extract from S01E02
        else:
            ep_num = int(episode_key.split('_')[1])  # Extract from episode_2
        
        print(f"\nProcessing Episode {ep_num} with {len(run_numbers)} runs...")
        
        for run_num in run_numbers:
            print(f"  Processing run {run_num}...")
            
            TASK_DIR = Path(f"dset/{sub_id}/ses-{ep_num:02d}/func") 
            
            # Construct the filename - note that run number is NOT zero-padded
            task_filename = f"{sub_id}_ses-{ep_num:02d}_task-strangerthings_run-{run_num}_part-mag_space-MNI152NLin2009cAsym_res-2_desc-preproc_bold.nii.gz"
            task_filepath = TASK_DIR / task_filename

            # Extract time series from 268 ROIs
            print(f"    Extracting time series from {task_filepath.name}...")
            time_series = caps_masker.fit_transform(task_filepath)
                
            print(f"    Original time series shape: {time_series.shape} (TRs x ROIs)")
            
            # Remove first 6 TRs for T1 equilibration
            if time_series.shape[0] > 6:
                time_series_trimmed = time_series[6:]  # Remove first 6 TRs
                print(f"    After removing first 6 TRs: {time_series_trimmed.shape} (TRs x ROIs)")
            else:
                print(f"    WARNING: Run has only {time_series.shape[0]} TRs, cannot remove 6 TRs!")
                time_series_trimmed = time_series  # Keep all TRs if less than 6
            
            # Track episode positions for each TR (after removing first 6)
            # Assuming each TR corresponds to one clip position sequentially
            # Add 6 to account for the removed TRs at the beginning of each run
            start_position = len(all_clip_positions) + 1 + 6
            tr_positions = list(range(start_position, start_position + time_series_trimmed.shape[0]))
            all_clip_positions.extend(tr_positions)
                
            # Store z-scored time series for this run (after TR removal)
            participant_timeseries.append(time_series_trimmed)

    if participant_timeseries:
        # Concatenate all runs for this participant
        print(f"\nConcatenating {len(participant_timeseries)} runs for {sub_id}...")
        participant_matrix = np.vstack(participant_timeseries)
        
        print(f"Original participant matrix shape: {participant_matrix.shape}")
        print(f"  - Total TRs across all runs: {participant_matrix.shape[0]}")
        print(f"  - Episode positions tracked: {len(all_clip_positions)}")
        
        # Create mask to exclude clips
        exclude_mask = np.array([pos in excluded_clips_ep2 for pos in all_clip_positions])
        keep_mask = ~exclude_mask
        
        excluded_count = np.sum(exclude_mask)
        print(f"  - Excluding {excluded_count} TRs based on removed clips")
        
        # Filter the participant matrix
        if np.any(keep_mask):
            participant_matrix_filtered = participant_matrix[keep_mask]
            print(f"Filtered participant matrix shape: {participant_matrix_filtered.shape}")
        else:
            print("Warning: All TRs would be excluded! Using original matrix.")
            participant_matrix_filtered = participant_matrix
        
        # Store the participant-level matrix
        all_participant_matrices[sub_id] = participant_matrix_filtered
        
        # Save the participant matrix
        output_dir = OUT_DIR / "weighted_timeseries"
        output_dir.mkdir(parents=True, exist_ok=True)
        
        output_file = output_dir / f"{sub_id}_ep2_CAP1_pos_weighted_timeseries.npy"
        np.save(output_file, participant_matrix_filtered)
        print(f"Saved participant matrix to: {output_file}")
        
        # Also save as CSV for easier inspection
        output_csv = output_dir / f"{sub_id}_ep2_CAP1_pos_weighted_timeseries.csv"
        df_matrix = pd.DataFrame(participant_matrix_filtered, 
                                columns=[f"CAP1_pos_weighted"])
        df_matrix.to_csv(output_csv, index=False)
        print(f"Saved participant matrix (CSV) to: {output_csv}")
        
        # Save exclusion info
        exclusion_info = {
            'original_trs_before_trimming': int(participant_matrix.shape[0]) + (len(participant_data[sub_id]["S01E02"]) * 6),  # Add back removed TRs
            'trs_removed_for_t1_equilibration': len(participant_data[sub_id]["S01E02"]) * 6,  # 6 TRs per run
            'original_trs_after_trimming': int(participant_matrix.shape[0]),
            'excluded_trs': int(excluded_count),
            'final_trs': int(participant_matrix_filtered.shape[0]),
            'excluded_positions': [float(pos) for pos in excluded_clips_ep2],
            'excluded_tr_indices': [int(idx) for idx in np.where(exclude_mask)[0].tolist()]
        }
        
        exclusion_file = output_dir / f"{sub_id}_ep2_exclusion_info.json"
        import json
        with open(exclusion_file, 'w') as f:
            json.dump(exclusion_info, f, indent=2)
        print(f"Saved exclusion info to: {exclusion_file}")
        


print(f"\n{'='*60}")
print("PROCESSING COMPLETE")
print(f"{'='*60}")
print(f"Processed {len(all_participant_matrices)} participants:")
for sub_id, matrix in all_participant_matrices.items():
    print(f"  {sub_id}: {matrix.shape[0]} TRs × {matrix.shape[1]} ROIs (after exclusions)")




Found 82 excluded clips for Episode 2: [1.0, 15.0, 32.0, 179.0, 180.0, 181.0, 182.0, 183.0, 184.0, 185.0]...

PROCESSING sub-Blossom

Processing Episode 2 with 7 runs...
  Processing run 1...
    Extracting time series from sub-Blossom_ses-02_task-strangerthings_run-1_part-mag_space-MNI152NLin2009cAsym_res-2_desc-preproc_bold.nii.gz...
[NiftiMapsMasker.wrapped] Loading regions from <nibabel.nifti1.Nifti1Image object at 0x11db61d00>
[NiftiMapsMasker.wrapped] Loading regions from <nibabel.nifti1.Nifti1Image object at 0x11db61d00>
[NiftiMapsMasker.wrapped] Resampling regions
[NiftiMapsMasker.wrapped] Resampling regions


/Users/chloehampson/Desktop/diva-emotion-dfc/.venv/lib/python3.14/site-packages/sklearn/utils/_set_output.py:316: UserWarning: memory_level is currently set to 0 but a Memory object has been provided. Setting memory_level to 1.
  data_to_wrap = f(self, X, *args, **kwargs)


[NiftiMapsMasker.wrapped] Finished fit
    Original time series shape: (318, 1) (TRs x ROIs)
    After removing first 6 TRs: (312, 1) (TRs x ROIs)
  Processing run 2...
    Extracting time series from sub-Blossom_ses-02_task-strangerthings_run-2_part-mag_space-MNI152NLin2009cAsym_res-2_desc-preproc_bold.nii.gz...
[NiftiMapsMasker.wrapped] Loading regions from <nibabel.nifti1.Nifti1Image object at 0x11db61d00>
    Original time series shape: (318, 1) (TRs x ROIs)
    After removing first 6 TRs: (312, 1) (TRs x ROIs)
  Processing run 2...
    Extracting time series from sub-Blossom_ses-02_task-strangerthings_run-2_part-mag_space-MNI152NLin2009cAsym_res-2_desc-preproc_bold.nii.gz...
[NiftiMapsMasker.wrapped] Loading regions from <nibabel.nifti1.Nifti1Image object at 0x11db61d00>
[NiftiMapsMasker.wrapped] Resampling regions
[NiftiMapsMasker.wrapped] Resampling regions
[NiftiMapsMasker.wrapped] Finished fit
[NiftiMapsMasker.wrapped] Finished fit
    Original time series shape: (255, 1) (TRs